# Assignment 12

### 1.复习上课内容

### 2.回答以下理论问题

#### 1. 请写一下TF-IDF的计算公式

去除停用词后, 计算一篇文章的每个词语的频率, 得到TF, 找一个很大的预料库, 用log(所有文章的数量 / 含有某词语的文章数量 + 1)得到每个词语的IDF, 再用TF * IDF获得某个词的值

#### 2. LDA算法的基本假设是什么？

每个文档的主题服从某一个分步, 每个主题的词语的频率服从某一个分部

#### 3. 在TextRank算法中构建图的权重是如何得到的？

通过反复迭代PR(Pi, t+1) = (1-d) + d * sigma(cosine((Pi, Pj) * RP(Pj)/sigma(cosine((Pj, Pk))), Pk∈out(Pj), 其中有个窗口大小的超参数, 如果窗口大小是5, 一句句子中的词语A,B,C,D,E,F,G,H,I,J,K,L,M,N, 和E相连的词语有A-I

#### 4. 什么是命名实体识别？ 有什么应用场景？

对一句话进行jieba词语的分割, 然后用seq2seq输出每个词的分类, 可以是通常的分类, 也可以是金融生物等领域的分类, 用于知识图谱, 对话理解, 舆情分析, 对话机器人

#### 5.NLP主要有哪几类任务 ？

分类和生成, 分类有情感分析, ner, 文本分类, 生成有机器翻译, 对话系统, 文本生成

### 3.实践题

#### 3.1 手动实现TextRank算法 (在新闻数据中随机提取100条新闻训练词向量和做做法测试）

 提示：
 1. 确定窗口，建立图链接。   
 2. 通过词向量相似度确定图上边的权重
 3. 根据公式实现算法迭代(d=0.85)

### 老师, 我觉得做词向量的时候不需要去掉停用词, 在用textrank的时候只要在最后计算完后获取最多词频的时候再把停用词去掉, 因为我觉得词语和词语之间的连接有的时候就是需要一些停用词放在中间, 如果去掉了停用词, textrank的窗口大小就相当于在去掉了停用词的地方变大了, 我这么理解有什么问题啊

In [34]:
import pandas as pd
import numpy as np
import gensim
import jieba
import re
import random
from gensim import models
from gensim import models
from gensim.models.word2vec import LineSentence, Word2Vec

In [261]:
with open(r'C:\Users\Nacho Liu\Desktop\assignment\data/article_9k.txt', 'r', encoding='utf8') as f:
    content = f.readlines()

In [262]:
window = 2
content = [list(jieba.cut(el)) for el in content]

In [263]:
with open(r'C:\Users\Nacho Liu\Desktop\assignment\data/百度停用词表.txt', 'r', encoding='utf8') as f:
    stop_words = f.readlines()

In [264]:
stop_words = [el.rstrip() for el in stop_words]
stop_words = set(stop_words)

In [265]:
content = [' '.join([e for e in el if e not in stop_words]) for el in content]

In [151]:
with open('gensim_input.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(content))

In [152]:
model = Word2Vec(corpus_file='gensim_input.txt', workers=8, sg=0, window=5, iter=5, compute_loss=True)

In [153]:
model.save('word2vec.model')

In [154]:
model.wv.most_similar('苹果')

[('开发者', 0.6919742822647095),
 ('诺基亚', 0.6833198070526123),
 ('iOS', 0.6659470200538635),
 ('小米', 0.6641458868980408),
 ('亚马逊', 0.6640409231185913),
 ('高通', 0.659730851650238),
 ('智能手机', 0.6515213847160339),
 ('App', 0.6509519815444946),
 ('音箱', 0.6237626671791077),
 ('谷歌', 0.6117413640022278)]

没有停用词的content

In [266]:
content = [el.split(' ') for el in content]

In [156]:
from collections import defaultdict
graph = defaultdict(set)

In [157]:
for sentence in content:
    ls = []
    length = len(sentence) - window + 1
    for i in range(length):
        ls.append(sentence[i: i + window])
        
    # 去除不在model.wv的无法计算cosine的词
    ls = [[e for e in el if e in model.wv] for el in ls]
    
    for el in ls:
        for index in range(len(el)):  # el ['本周', '6']
            graph[el[index]].update(set(el[:index] + el[index + 1:]))
    

In [158]:
graph['再次']

{'格局',
 '便',
 '获批',
 '禅城',
 '抢下',
 '丝绸之路',
 '深铁',
 '达沃斯',
 '此次',
 '购票',
 '声音',
 '工作',
 '三点',
 '挺进',
 '移民',
 '上机',
 '翻出',
 '发射台',
 '蓝军',
 '蒙',
 '怀孕',
 '接到',
 '当下',
 '电视台',
 '败给',
 '1709',
 '求医',
 '爬起来',
 '射手',
 '证券时报',
 '台湾同胞',
 '差距',
 '凯塔',
 '联赛',
 '全国',
 '陈女士',
 '核试验',
 '退场',
 '报名',
 '走上',
 '探访',
 '民事',
 '暴露出',
 '污染',
 '年',
 '加码',
 '实力',
 '录得',
 '学校',
 '审判',
 '发',
 '第二',
 '站上',
 '我会',
 '西班牙',
 '会晤',
 '冲向',
 '一席话',
 '年来',
 '右下角',
 '赛里木湖',
 '发飙',
 '改善',
 '摩托车',
 '放弃',
 '旧账',
 '大喊',
 '周琦',
 '两队',
 '大脑',
 '债务',
 '消沉',
 '遏制',
 '盘中',
 '李浩',
 '关注',
 '唱',
 '漂着',
 '荣获',
 '今夏',
 '讲话',
 '鼓吹',
 '地位',
 '提请',
 '启用',
 '尼奥斯',
 '器具',
 '儿子',
 '呼声',
 '煞',
 '出演',
 '更新',
 '交织',
 '罗纳尔迪',
 '开庭',
 '大使',
 '瓷窑',
 '问及',
 '崛起',
 '地球',
 '忘怀',
 '抓住',
 '阳光城',
 '就此',
 '股价',
 '申请人',
 '课纲',
 '地步',
 '滑雪场',
 '畸形',
 '占据',
 '上前',
 '期',
 '流亡',
 '制造',
 '瞬间',
 '当天',
 '本周',
 '汇款',
 '当地政府',
 '特梅尔',
 '效仿',
 '远高于',
 '扩容',
 '第四名',
 '热情',
 '美军',
 '试射',
 '赴',
 '不禁',
 '企业',
 '巴萨',
 '驾驶',
 '攀高',
 '迫近',
 '一个',
 '托起',
 '捐赠',
 '管委会',
 '

In [159]:
len(graph)

99062

In [160]:
word_dict = {el: 1/len(graph) for el in list(graph.keys())}

In [161]:
word_dict['再次']

1.0094688175082272e-05

In [183]:
from scipy.spatial.distance import cosine

def get_similarity(word1, word2):
    return cosine(model.wv[word1], model.wv[word2])

In [186]:
word_dict_cosine = {}
for i, word in enumerate(list(graph.keys())):
    for w in graph[word]:
        if f'{word}_{w}' not in word_dict_cosine:
            word_dict_cosine[f'{word}_{w}'] = get_similarity(word, w)
            word_dict_cosine[f'{w}_{word}'] = word_dict_cosine[f'{word}_{w}']
    if i % 1000 == 0:
        print(f'\r{i}', end='')

99000

In [188]:
word_dict_cosine_sum = {}
for i, word in enumerate(list(graph.keys())):
    word_dict_cosine_sum[word] = sum([word_dict_cosine[f'{word}_{w}'] for w in graph[word]])
    if i % 1000 == 0:
        print(f'\r{i}', end='')

99000

In [192]:
word_dict_cosine_sum['本周'], word_dict_cosine_sum['特朗普'], word_dict_cosine_sum['唐纳德']

(410.8926269665826, 1804.1121834403657, 68.21052476693876)

In [203]:
d = 0.85
for epoch in range(5):
    for i, (word, tr) in enumerate(word_dict.items()):
        linked_word_set = graph[word]
        num = 0
        for w in linked_word_set:  # '再次'
            num += get_similarity(word, w) * word_dict[w] / word_dict_cosine_sum[w]

        word_dict[word] = (1 - d) + d * num

        if i % 100 == 0:
            print(f'\r{epoch}_{i}      ', end='')

4_99000

In [130]:
# 1 epoch
word_dict

{'本周': 1.009479007884031e-05,
 '6': 1.009479007884031e-05,
 '月': 1.009479007884031e-05,
 '12': 1.009479007884031e-05,
 '日起': 1.009479007884031e-05,
 '小米': 1.009479007884031e-05,
 '手机': 1.009479007884031e-05,
 '15': 1.009479007884031e-05,
 '款': 1.009479007884031e-05,
 '机型': 1.009479007884031e-05,
 '外': 1.009479007884031e-05,
 '已': 1.009479007884031e-05,
 '暂停': 1.009479007884031e-05,
 '更新': 1.009479007884031e-05,
 '发布': 1.009479007884031e-05,
 '含': 1.009479007884031e-05,
 '开发': 1.009479007884031e-05,
 '版': 1.009479007884031e-05,
 '内测': 1.009479007884031e-05,
 '影响': 1.009479007884031e-05,
 '确保': 1.009479007884031e-05,
 '工程师': 1.009479007884031e-05,
 '精力': 1.009479007884031e-05,
 '工作': 1.009479007884031e-05,
 '有人': 1.009479007884031e-05,
 '猜测': 1.009479007884031e-05,
 '用到': 1.009479007884031e-05,
 '研发': 1.009479007884031e-05,
 '之中': 1.009479007884031e-05,
 '去年': 1.009479007884031e-05,
 '5': 1.009479007884031e-05,
 '距今已有': 1.009479007884031e-05,
 '一年': 1.009479007884031e-05,
 '有余': 1.009479

In [202]:
word_dict

{'本周': 0.15004805523113285,
 '6': 0.15139797634746766,
 '月': 0.15145338866900482,
 '12': 0.1503565506752869,
 '日起': 0.15017389695954647,
 '小米': 0.1503749441440722,
 '手机': 0.15058240538933865,
 '15': 0.15071532808992058,
 '款': 0.1506301748205605,
 '机型': 0.15062149557525817,
 '外': 0.15150155945864466,
 '已': 0.1542668320289687,
 '暂停': 0.15050964911950954,
 '更新': 0.15031339321064568,
 '发布': 0.15203369985672238,
 '含': 0.15040280280898186,
 '开发': 0.15099151290165125,
 '版': 0.15049580785011624,
 '内测': 0.15000019593521138,
 '影响': 0.15136795884561666,
 '确保': 0.1506010966817167,
 '工程师': 0.15024441378516704,
 '精力': 0.1504139480008704,
 '工作': 0.153979146333846,
 '有人': 0.15074876446109167,
 '猜测': 0.15072744687745612,
 '用到': 0.1505042574934824,
 '研发': 0.15086378374463438,
 '之中': 0.1504195387964318,
 '去年': 0.17408989217823667,
 '5': 0.15432220757017506,
 '距今已有': 0.15006363509152043,
 '一年': 0.1609537449989764,
 '有余': 0.1500862521584652,
 '更新换代': 0.15423075502181774,
 '确切': 0.1500204695391984,
 '信息': 0

In [204]:
# 6 epoch
word_dict

{'本周': 4.757984757404134,
 '6': 62.425326601200084,
 '月': 74.08147378750108,
 '12': 23.08520502151182,
 '日起': 4.936677175121516,
 '小米': 2.631144232451122,
 '手机': 18.01151283495106,
 '15': 22.230925123415947,
 '款': 5.373354582196072,
 '机型': 1.556684798571228,
 '外': 27.42563600837689,
 '已': 105.90632396838154,
 '暂停': 5.622300107920591,
 '更新': 5.111563153494533,
 '发布': 28.621506214407688,
 '含': 5.871162016443237,
 '开发': 16.490056975312775,
 '版': 6.680862019648139,
 '内测': 0.2062840794531308,
 '影响': 45.40262437689019,
 '确保': 21.380665958719025,
 '工程师': 4.038391863518351,
 '精力': 1.8270348963421923,
 '工作': 81.5942891887768,
 '有人': 14.226919124350916,
 '猜测': 2.148483545669816,
 '用到': 0.5900923229741933,
 '研发': 14.267889276745171,
 '之中': 6.867526660280733,
 '去年': 41.66239068322043,
 '5': 72.18398953388751,
 '距今已有': 0.30903615238715754,
 '一年': 21.56408045648509,
 '有余': 0.4866864823661514,
 '更新换代': 0.5069617669044342,
 '确切': 0.7881532825714254,
 '信息': 31.42290108948809,
 '等待': 8.54919449500153,
 

In [208]:
sorted([(k, v) for k, v in word_dict.items()], key=lambda x: x[1], reverse=True)[:100].__str__()

"[('n', 800.840555387216), ('说', 221.50927389983997), ('中', 216.49322985009871), ('上', 195.2757923481608), ('中国', 181.1226127313262), ('都', 156.79285126035697), ('nn', 155.8463497826832), ('后', 145.86049745999102), ('还', 135.98451070099412), ('人', 131.9400340997407), ('２', 131.4825430188395), ('记者', 129.7894113471982), ('１', 125.14027510899932), ('年', 124.81701681094432), ('一个', 123.92373457435899), ('不', 123.41509829352155), ('日', 119.71361721274889), ('已', 105.90632396838154), ('时', 100.65568816540431), ('会', 97.20567082926975), ('nnn', 91.8822090511267), ('下', 82.8830052092383), ('工作', 81.5942891887768), ('美国', 81.51796667569366), ('企业', 80.7100809975007), ('发展', 78.62610549787952), ('国家', 76.52971453785912), ('更', 76.4302605724439), ('新', 75.4994104487002), ('完', 75.40779186746222), ('前', 75.07812302141595), ('公司', 74.93097803920915), ('３', 74.40869368928821), ('月', 74.08147378750108), ('大', 72.22454220394803), ('5', 72.18398953388751), ('５', 70.43622771303288), ('项目', 68.439884929

### 3.2 使用词向量和k-means的方法寻找关键词

提示：   
1.使用3.1训练好的词向量   
2.可使用sklearn等机器学习库

In [209]:
from sklearn.cluster import KMeans

In [220]:
model.similar_by_vector(model.wv['再次'])

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similar_by_vector` (Method will be removed in 4.0.0, use self.wv.similar_by_vector() instead).
  """Entry point for launching an IPython kernel.


[('再次', 1.0),
 ('再度', 0.6120811104774475),
 ('第二次', 0.6091792583465576),
 ('终于', 0.5341813564300537),
 ('三次', 0.532611608505249),
 ('此后', 0.5255109071731567),
 ('随后', 0.5138967037200928),
 ('历史性', 0.5095598101615906),
 ('就此', 0.5056745409965515),
 ('如期', 0.49575597047805786)]

In [233]:
vocab = list(model.wv.vocab.keys())

In [234]:
data = [model.wv[w] for w in vocab]

In [211]:
clusters = 5

In [236]:
kmeans = KMeans(n_clusters=clusters, random_state=0).fit(data)

In [238]:
kmeans.cluster_centers_.shape

(5, 100)

In [240]:
for index, vec in enumerate(kmeans.cluster_centers_):
    print(index, model.similar_by_vector(vec, topn=10), end='\n\n')

0 [('拉多布库', 0.965475857257843), ('坎蒂略', 0.9639393091201782), ('稻富', 0.9599762558937073), ('手掌心', 0.9584068059921265), ('山楂糕', 0.95391446352005), ('什前', 0.9534969329833984), ('脯', 0.9534062147140503), ('查瓦', 0.9528214335441589), ('一贯方针', 0.9517797231674194), ('cdsb86612222', 0.9513211846351624)]

1 [('卡尔', 0.7867547273635864), ('路易斯', 0.7844663858413696), ('斯在', 0.7746326923370361), ('穆斯', 0.7705637812614441), ('奇', 0.763282060623169), ('洛夫', 0.7618637084960938), ('布罗', 0.7618622779846191), ('坎特', 0.7600472569465637), ('塔诺', 0.7598129510879517), ('卡特', 0.7580133676528931)]

2 [('长发', 0.8897824287414551), ('黝黑', 0.8778571486473083), ('深深地', 0.8733784556388855), ('唱起', 0.8702466487884521), ('背带', 0.8664035797119141), ('艳丽', 0.8648862242698669), ('文姬', 0.8629333972930908), ('昏暗', 0.8624947667121887), ('淡淡的', 0.8618084788322449), ('一副', 0.856205940246582)]

3 [('着眼于', 0.8157681822776794), ('力求', 0.7620406746864319), ('迫切需要', 0.7580281496047974), ('重要性', 0.7494403123855591), ('决策者', 0.748754

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `similar_by_vector` (Method will be removed in 4.0.0, use self.wv.similar_by_vector() instead).
  


### 3.3  提取新闻人物里的对话。(使用以上提取小数据即可）

提示：    
1.寻找预料里具有表示说的意思。    
2.使用语法分析提取句子结构。    
3.检测谓语是否有表示说的意思。

我python37装不了pyltp

In [259]:
model.similar_by_word('说', topn=30)

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similar_by_word` (Method will be removed in 4.0.0, use self.wv.similar_by_word() instead).
  """Entry point for launching an IPython kernel.


[('指出', 0.7242071032524109),
 ('告诉', 0.6779590845108032),
 ('介绍', 0.6721910238265991),
 ('称', 0.6179640889167786),
 ('说完', 0.6166958808898926),
 ('明说', 0.5950311422348022),
 ('坦言', 0.5557621121406555),
 ('文说', 0.544553816318512),
 ('透露', 0.535801112651825),
 ('地说', 0.5331635475158691),
 ('中说', 0.519256591796875),
 ('对此', 0.513698935508728),
 ('所说', 0.49106431007385254),
 ('问', 0.46522367000579834),
 ('提到', 0.4454174041748047),
 ('说法', 0.44170984625816345),
 ('道', 0.43249595165252686),
 ('说据', 0.42463603615760803),
 ('言', 0.41562420129776),
 ('谈到', 0.41456931829452515),
 ('建议', 0.4125736951828003),
 ('说道', 0.40602460503578186),
 ('直言', 0.4011193811893463),
 ('讲', 0.39992356300354004),
 ('时说', 0.39354223012924194),
 ('很大', 0.3918382227420807),
 ('事情', 0.3885289132595062),
 ('为此', 0.387066513299942),
 ('呼吁', 0.3864017724990845),
 ('希望', 0.38582804799079895)]

In [242]:
import jieba.posseg as jp

In [245]:
sentence = '我爱Python数据分析'
posseg = jp.cut(sentence)
for i in posseg:
    print(i.__dict__)

{'word': '我', 'flag': 'r'}
{'word': '爱', 'flag': 'v'}
{'word': 'Python', 'flag': 'eng'}
{'word': '数据分析', 'flag': 'l'}


In [267]:
c = [''.join([e for e in el if e not in stop_words]) for el in content]

In [268]:
c = [[i.__dict__ for i in jp.cut(el)] for el in c[:100]]

In [270]:
c[:2]

[[{'word': '本周', 'flag': 't'},
  {'word': '6', 'flag': 'm'},
  {'word': '月', 'flag': 'm'},
  {'word': '12', 'flag': 'm'},
  {'word': '日', 'flag': 'm'},
  {'word': '起', 'flag': 'v'},
  {'word': '小米', 'flag': 'n'},
  {'word': '手机', 'flag': 'n'},
  {'word': '615', 'flag': 'm'},
  {'word': '款', 'flag': 'm'},
  {'word': '机型', 'flag': 'n'},
  {'word': '外', 'flag': 'f'},
  {'word': '机型', 'flag': 'n'},
  {'word': '已', 'flag': 'd'},
  {'word': '暂停', 'flag': 'd'},
  {'word': '更新', 'flag': 'd'},
  {'word': '发布', 'flag': 'v'},
  {'word': '含', 'flag': 'v'},
  {'word': '开发', 'flag': 'v'},
  {'word': '版', 'flag': 'n'},
  {'word': '体验版', 'flag': 'n'},
  {'word': '内测', 'flag': 'v'},
  {'word': '稳定版', 'flag': 'n'},
  {'word': '暂不', 'flag': 'd'},
  {'word': '受', 'flag': 'v'},
  {'word': '影响', 'flag': 'vn'},
  {'word': '确保', 'flag': 'v'},
  {'word': '工程师', 'flag': 'n'},
  {'word': '精力', 'flag': 'n'},
  {'word': '系统优化', 'flag': 'n'},
  {'word': '工作', 'flag': 'vn'},
  {'word': '有人', 'flag': 'r'},
  {'word':

In [255]:
model.similarity('说', '适合'), model.similarity('说', '指出'), 

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.


(0.20806926, 0.7242071)

In [273]:
threshold = 0.3999
for sentence in c:
    label = False
    for word in sentence:
        if word['flag'] == 'v' and word['word'] in model.wv and model.similarity(word['word'], '说') > threshold:
            label = True
            break
            
    if label: 
        print(''.join([el['word'] for el in sentence]))

骁龙835唯一Windows10桌面平台认证ARM处理器高通只性能去屏蔽掉小核心正联手微软找到一种适合桌面平台兼顾性能功耗完美方案报道称微软拿到新源码Windows10更好理解biglittle架构资料显示骁龙835一款集成CPUGPU基带蓝牙WiFiSoC传统Wintel方案节省至少30PCB空间按计划Q4华硕惠普联想首发骁龙835Win10电脑预计均二合一形态产品高通骁龙未来也许还见到三星Exynos联发科华为麒麟小米澎湃Windows10桌面平台

原标题44岁女子跑深圳约会网友拒暴雨中裸身奔走深圳交警微博称昨日清晨交警发现女子赤裸上身行走南坪快速上期间还轻生年头一辅警发现后赶紧披上黄衣一路劝说事发时到底都发生些南都记者带还原现场南都记者龙岗大队坂田中队见到辅警刘青发现女生辅警一位外表高大帅气说话略带些腼腆90后青年刘青介绍6月16日早上7时36分环城南路值勤接到中队一位女子裸身机动车危险警情随后骑着小铁骑沿路寻找大概花十多分钟南坪大道坂田出口龙岗方向逆行辅道上发现女子女子身上一丝不挂地逆车流行时走时停时坐时躺险象环生刘青停好小铁骑一名巡防员追发现女子情绪很低落话不刘青尝试女子交流劝说女子离开女子不接受缓慢地往南坪快速路主干道上走去路边上聚集很市民围观不刺激女子情绪刘青巡防员盯女子驱赶围观群众现场还原警方提供一份视频16日早上7时25分女子坂雪岗大道环城南路监控视频中女子还穿着白色内裤正坂雪岗大道南坪快速方向缓慢走正值上班高峰期十字路口车流排起长队女子时路上市民纷纷驻足观望不少车辆放慢速度女子不市民观望停下脚步依然缓慢走女子行进十字路口时一辆大货车挡住镜头女子再次出现镜头时发现女子没穿内裤全身裸露南坪快速方向走去记者发现视频中女子人尾随上前劝止市民大清早路上情况恐怕都没办法淡定面对这一情况刘青看挺别扭感觉很尴尬刘青跟随女子上南坪快速路主干道时女子作出人意想不到举动靠近护栏跳下去刘青赶忙冲上去拉住女子手控制住远离护栏碍于女子穿衣服刘青递上衣服女子没接受还衣服扔排水沟里往前走没办法刘青只能紧紧拉一只手刘青一路上耐心开导安慰只听到重复一句话遭遇事会期间还不时试图挣脱刘青手要冲向护栏往下跳牵走大概十多分钟天下起大暴雨雨大眼睛都睁不开刘青说瞬间雨透女子依然不接受刘青大雨往前走大概走四十分钟女子停下来说想回家女子接受刘青递过来小黄衣深圳微博上照片女子披着小黄衣刘青小心翼翼

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """
